In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/bootcamp/Averiguacao Cadastral/Dados

/content/drive/MyDrive/bootcamp/Averiguacao Cadastral/Dados


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Carga de Bases de Dados

In [ ]:
# Load the datasets
df_pessoas_final = pd.read_csv('df_pessoas_final.csv')
df_familia_final =  pd.read_csv('df_familia_final.csv')

## Preparação de Atributos - Características da família e Composição familiar

In [ ]:
import numpy as np
df_familia_final['ind_parc_mds_fam'] = np.where(df_familia_final['ind_parc_mds_fam'] != 0, 1, 0)

In [ ]:
# Agrupar os dados de pessoas por família e aplicar as funções de agregação
df_pessoas_agrupado_composicao_familiar = df_pessoas_final.groupby('id_familia').agg(
    # Contando o número de pessoas por família
    qtd_pessoas=('id_familia', 'size'),
    # Verificando se há pessoas em cada faixa etária por família
    infancia=('idade', lambda x: 1 if sum(1 for i in x if 0 <= i <= 6) > 0 else 0),
    crianca_adolescente=('idade', lambda x: 1 if sum(1 for i in x if 7 <= i <= 17) > 0 else 0),
    adultos=('idade', lambda x: 1 if sum(1 for i in x if 18 <= i <= 59) > 0 else 0),
    idoso=('idade', lambda x: 1 if sum(1 for i in x if i >= 60) > 0 else 0),
    # Verificando se há pessoas com deficiência na família
    deficiencia=('cod_deficiencia_memb', lambda x: 1 if any(x != 1) else 0)
)

# Renomear as colunas e resetar o índice
df_pessoas_agrupado_composicao_familiar = df_pessoas_agrupado_composicao_familiar.reset_index()
df_pessoas_agrupado_composicao_familiar = df_pessoas_agrupado_composicao_familiar[['id_familia','qtd_pessoas', 'infancia', 'crianca_adolescente','adultos','idoso','deficiencia' ]]


##Preparação de Atributos - Escolaridade

In [ ]:
# Nova variável 'alfabetizado'
df_agrupado_escolaridade = df_pessoas_final.copy()
df_agrupado_escolaridade['alfabetizado'] = df_agrupado_escolaridade.apply(lambda row: 1 if row['idade'] > 10 and row['cod_sabe_ler_escrever_memb'] == 1 else 0, axis=1)

# Nova variável 'frequenta_escola'
df_agrupado_escolaridade['frequenta_escola'] = df_agrupado_escolaridade.apply(lambda row: 1 if row['idade'] < 17 and row['ind_frequenta_escola_memb'] <= 2 else 0, axis=1)

# Nova variável 'frequenta_escola_publica'
df_agrupado_escolaridade['frequenta_escola_publica'] = df_agrupado_escolaridade.apply(lambda row: 1 if row['idade'] < 17 and row['ind_frequenta_escola_memb'] == 1 else 0, axis=1)

# Nova variável 'frequenta_escola_privada'
df_agrupado_escolaridade['frequenta_escola_privada'] = df_agrupado_escolaridade.apply(lambda row: 1 if row['idade'] < 17 and row['ind_frequenta_escola_memb'] == 2 else 0, axis=1)

# Nova variável 'frequenta_escola_nunca'
df_agrupado_escolaridade['frequenta_escola_nunca'] = df_agrupado_escolaridade.apply(lambda row: 1 if row['idade'] < 17 and row['ind_frequenta_escola_memb'] == 3 else 0, axis=1)

# Nova variável 'frequenta_escola_nunca_adulto'
df_agrupado_escolaridade['frequenta_escola_nunca_adulto'] = df_agrupado_escolaridade.apply(lambda row: 1 if row['idade'] >= 18 and row['ind_frequenta_escola_memb'] == 3 else 0, axis=1)

# Agrupando por 'id_familia' e somando as novas variáveis
df_agrupado_escolaridade = df_agrupado_escolaridade.groupby('id_familia').max().reset_index()
df_agrupado_escolaridade = df_agrupado_escolaridade[['alfabetizado','frequenta_escola','frequenta_escola_publica','frequenta_escola_nunca', 'frequenta_escola_privada', 'frequenta_escola_nunca_adulto','id_familia' ]]

##Preparação de Atributos - Trabalho

In [ ]:
df_agrupado_trabalho = df_pessoas_final.copy()

# Nova variável 'trabalho_semana_adulto'
df_agrupado_trabalho['trabalho_semana_adulto'] = df_agrupado_trabalho.apply(lambda row: 1 if 18 <= row['idade'] <= 59 and row['cod_trabalhou_memb'] == 1 else 0, axis=1)

# Nova variável 'trabalho_semana_idoso'
df_agrupado_trabalho['trabalho_semana_idoso'] = df_agrupado_trabalho.apply(lambda row: 1 if row['idade'] >= 60 and row['cod_trabalhou_memb'] == 1 else 0, axis=1)

# Nova variável 'trabalho_semana_criança'
df_agrupado_trabalho['trabalho_semana_criança'] = df_agrupado_trabalho.apply(lambda row: 1 if row['idade'] < 18 and row['cod_trabalhou_memb'] == 1 else 0, axis=1)

# Nova variável 'afastado_trabalho'
df_agrupado_trabalho['afastado_trabalho'] = df_agrupado_trabalho.apply(lambda row: 1 if 18 <= row['idade'] <= 59 and row['cod_afastado_trab_memb'] == 1 else 0, axis=1)

# Nova variável 'trabalho_principal'
df_agrupado_trabalho['trabalho_principal'] = df_agrupado_trabalho.apply(lambda row: 1 if 18 <= row['idade'] <= 59 and row['cod_principal_trab_memb'] == 1 else 0, axis=1)

# Nova variável 'trabalho_12meses_adulto'
df_agrupado_trabalho['trabalho_12meses_adulto'] = df_agrupado_trabalho.apply(lambda row: 1 if 18 <= row['idade'] <= 59 and row['cod_trabalho_12_meses_memb'] == 1 else 0, axis=1)

# Nova variável 'trabalho_12meses_idoso'
df_agrupado_trabalho['trabalho_12meses_idoso'] = df_agrupado_trabalho.apply(lambda row: 1 if row['idade'] >= 60 and row['cod_trabalho_12_meses_memb'] == 1 else 0, axis=1)

# Nova variável 'trabalho_12meses_criança'
df_agrupado_trabalho['trabalho_12meses_criança'] = df_agrupado_trabalho.apply(lambda row: 1 if row['idade'] < 18 and row['cod_trabalho_12_meses_memb'] == 1 else 0, axis=1)

# Nova variável 'meses_trabalho'
df_agrupado_trabalho['meses_trabalho'] = df_agrupado_trabalho.apply(lambda row: row['qtd_meses_12_meses_memb'] if row['idade'] >= 18 else 0, axis=1)

# Agrupando por 'id_familia' e somando as novas variáveis
df_agrupado_trabalho = df_agrupado_trabalho.groupby('id_familia').max().reset_index()

df_agrupado_trabalho = df_agrupado_trabalho[['trabalho_semana_adulto','trabalho_semana_idoso','trabalho_semana_criança', 'afastado_trabalho','trabalho_principal','trabalho_12meses_adulto','trabalho_12meses_idoso', 'trabalho_12meses_criança','meses_trabalho','id_familia' ]]


## Unindos os dataframes

In [ ]:
# Unir os dataframes
df_pessoas_final_rf = df_pessoas_final[df_pessoas_final['cod_parentesco_rf_pessoa'] == 1]
#df_pessoas_final = df_pessoas_final.drop(['cod_parentesco_rf_pessoa'], axis=1)
df_pessoas_final_rf = pd.merge(df_pessoas_final_rf, df_pessoas_agrupado_composicao_familiar, on='id_familia', how='inner')
df_pessoas_final_rf = pd.merge(df_pessoas_final_rf, df_agrupado_escolaridade, on='id_familia', how='inner')
df_pessoas_final_rf = pd.merge(df_pessoas_final_rf, df_agrupado_trabalho, on='id_familia', how='inner')



In [ ]:
df_completo_rf = pd.merge(df_familia_final, df_pessoas_final_rf, on='id_familia', how='inner')


In [ ]:
# Salvando os arquivos finais de df_familia_final e df_pessoa_final
df_completo_rf.to_csv('df_familia_pessoas_final.csv', index=False)